 <p style= "background-color:#f6a85e;font-family:Arial;color:#FFFFFF;font-size:200%;text-align:center;border-radius:20px 20px;border-style:solid;border-width:10px;border-color:#f6a85e;"><b>Web Scraping</b></p>

**_Pablo Arriagada Ojeda_ | 2023** 

#### Autos de la marca Mazda en venta www.chileautos.cl

En este proyecto de web scraping, intentaremos sacar datos de la página de ventas de autos usados 'www.chileautos.cl'. El objetivo final es obtener un DataFrame con la información de todos los autos marca Mazda disponibles en la página a la fecha (Septiembre 19, 2023).

Como son muchas las publicaciones que maneja la página, con objetivos educativos, solamente obtendremos los datos de una marca. 

El proceso comienza con el Scraping de un a publicación, para así obtener las ubicaciones de los elementos que solicitaremos en un proceso automatizado.

In [1]:
#El proceso comienza cargando librerías.

import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import datetime
import traceback

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

# Scraping de la publicación.

In [11]:
#Abre la publicación de un solo auto
#y parsea su código HTML

browser = uc.Chrome()
url = 'https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-2-5-skyactiv-g-gt-auto-a-bose/CL-AD-15813972/?Cr=0&gts=CL-AD-15813972&gtsSaleId=CL-AD-15813972&gtsViewType=showcase&rankingType=showcase'
browser.get(url)
browser.implicitly_wait(10)
html = browser.page_source
soup = bs(html, 'html.parser')
print(soup)

## Obtener las variables.
### Varios datos con la misma clase.

A veces, hay datos que están distribuidos usando la misma clase, por lo que se desarrolló una función que itera sobre los elementos contenidos en la clase y los agrega a una lista.

In [19]:
def obtener_texto(clase):
    elementos = soup.find_all(class_= clase)
    lista= []
    for elemento in elementos:
        texto = elemento.get_text()
        lista.append(texto)
    
    print(f'{lista}\n')
    return lista

In [18]:
#Obtener el título del anuncio

titulo = obtener_texto('col-lg-8 col-sm-10')
titulo = titulo[0].strip()
print(f'El título del anuncio es: {titulo}')

['\n 2018 Mazda 3 2.5 Skyactiv-G GT Auto A. Bose \n']

El título del anuncio es: 2018 Mazda 3 2.5 Skyactiv-G GT Auto A. Bose


In [28]:
#Obtener el título del anuncio para la url

titulo_minus= titulo.strip().replace('.',' ')
titulo_minus= re.sub(r'\s+', ' ', titulo_minus) 
titulo_minus= titulo_minus.replace(' ', '-').lower()

print(titulo_minus)

2018-mazda-3-2-5-skyactiv-g-gt-auto-a-bose


In [45]:
#Obtener los detalles principales del anuncio almacenados en una tabla

detalles = obtener_texto('key-details-item-title')
km = detalles[0].replace(' km ', '').replace(',','')
transmision = detalles[1].strip()
combustible = detalles[2].strip()

print(f'Kilometraje: {km} km.\nTransmisión: {transmision} \nCombustible: {combustible}')

['50,700 km ', 'Automática ', 'Bencina ']

Kilometraje: 50700 km.
Transmisión: Automática 
Combustible: Bencina


In [51]:
#Obtener el precio y formatearlo como int

precio = obtener_texto('col features-item-value features-item-value-precio')
precio = precio[0].strip()
precio = int(precio.replace('$','').replace(',','').replace(' CLP',''))

print(f'El precio publicado es: {precio}')

['\n        $15,890,000 CLP\n    ']

El precio publicado es: 15890000


In [63]:
#Obtener la marca

marca = obtener_texto('col features-item-value features-item-value-marca')
marca = marca[0].strip().title()

print(f'El auto es marca {marca}')

['\n        MAZDA\n    ', '\n        MAZDA\n    ']

El auto es marca Mazda


In [64]:
#Obtener el modelo

modelo = obtener_texto('col features-item-value features-item-value-modelo')
modelo = modelo[0].strip().title()

print(f'El auto marca {marca} es modelo {modelo}')

['\n        MAZDA3\n    ', '\n        MA3\n    ']

El auto marca Mazda es modelo Mazda3


In [65]:
#Obtener capacidad de motor

litros_motor = obtener_texto('col features-item-value features-item-value-motor-litros')
litros_motor = float(litros_motor[0].strip())

print(f'Tiene un motor tamaño {litros_motor} litros')

['\n        2.5\n    ']

Tiene un motor tamaño 2.5 litros


In [67]:
#Obtener la ubicación del vendedor

ubicacion = obtener_texto('mb-1')
ubicacion = ubicacion[0].strip()

print(f'El vendedor es de {ubicacion}')

['\n            Viña del Mar, Valparaíso\n', '\n            Viña del Mar, Valparaíso\n']

El vendedor es de Viña del Mar, Valparaíso


In [72]:
#Crea un dataframe para el auto parseado

df_auto = pd.DataFrame({'Título': titulo,
                       'Marca': marca,
                       'Modelo': modelo,
                       'Kilometraje': km,
                       'Ubicación': ubicacion,
                       'Precio': precio,
                        'Transmisión': transmision
                       }, index= [0])
df_auto

,Título,Marca,Modelo,Kilometraje,Ubicación,Precio,Transmisión
0,2018 Mazda 3 2.5 Skyactiv-G GT Auto A. Bose,Mazda,Mazda3,50700,"Viña del Mar, Valparaíso",15890000,Automática


# Automatización del proceso.
Como haremos muchas consultas a una página web, lo ideal es ir rotando nuestra dirección IP, con tal de no ser bloqueados y lograr con éxito el scraping.

In [57]:
#Ingresa a la página principal de www.chileautos.cl y accede al filtro por marca Mazda

browser = uc.Chrome()
url = 'https://www.chileautos.cl/vehiculos/mazda/'
browser.get(url)
browser.implicitly_wait(10)
html = browser.page_source

In [58]:
#Toma el código HTML y lo parsea

soup = bs(html, 'html.parser')
print(soup)

<html lang="es-CL"><head><style>body {transition: opacity ease-in 0.2s; } 
body[unresolved] {opacity: 0; display: block; overflow: hidden; position: relative; } 
</style><style class="vjs-styles-defaults">
      .video-js {
        width: 300px;
        height: 150px;
      }

      .vjs-fluid {
        padding-top: 56.25%
      }
    </style>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no, shrink-to-fit=no" name="viewport"/>
<script src="https://js-agent.newrelic.com/nr-spa-1216.min.js"></script><script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/443166369570098?v=2.9.127&amp;r=stable&amp;domain=www.chileautos.cl"></script><script async="" src="https://connect.facebook.net/signals/config/1072570

## Búsqueda de publicaciones
Este código itera en la página buscando las ID y enlaces de cada publicación, para más adelante poder acceder una a una y obtener los datos de cada publicación.

In [62]:
contador_url = 1
contador_pagina = 1

busqueda = ''
offset = 0
contador_pagina = 1
ids = []
links = []

while True:
    url_actual = f"https://www.chileautos.cl/vehiculos/mazda/?offset={offset}"

    browser.get(url_actual)

    time.sleep(random.randint(5, 8))

    html = browser.page_source
    soup = bs(html, 'html.parser')

    pagina_actual = int(soup.find('span', {'class': 'page-link'}).text.strip().replace('(current)', ''))

    if contador_pagina == pagina_actual:

        publicaciones = soup.find_all('div', class_='listing-item card showcase')
        enlaces = soup.find_all('a', {'class': 'js-encode-search view-more view-more-gallery'})

        if not enlaces:
            break

        for i, publicacion in enumerate(publicaciones):
            id_autos = publicacion.get('id')
            enlace_publicacion = enlaces[i].get('href')

            if enlace_publicacion:
                enlace_completo = "https://www.chileautos.cl" + enlace_publicacion
                ids.append(id_autos)
                links.append(enlace_completo)

                time.sleep(random.randint(1, 3))
                print(f'ID: {id_autos}\nEnlace: {enlace_completo}')
            
    else:
        break

ID: CL-AD-14735282
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-6-2-0-skyactiv-g-le-auto/CL-AD-14735282/?Cr=0&gts=CL-AD-14735282&gtsSaleId=CL-AD-14735282&gtsViewType=showcase&rankingType=showcase
ID: CL-AD-15499247
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-15499247/?Cr=0&gts=CL-AD-15499247&gtsSaleId=CL-AD-15499247&gtsViewType=showcase&rankingType=showcase
ID: CL-AD-15185232
Enlace: https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-4wd/CL-AD-15185232/?Cr=0&gts=CL-AD-15185232&gtsSaleId=CL-AD-15185232&gtsViewType=showcase&rankingType=showcase
ID: CL-AD-15158840
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-3-sport/CL-AD-15158840/?Cr=0&gts=CL-AD-15158840&gtsSaleId=CL-AD-15158840&gtsViewType=showcase&rankingType=showcase
ID: CP-AD-8267126
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-2-1-5-v/CP-AD-8267126/?Cr=0
ID: CP-AD-8272121
Enlace: https:

ID: CP-AD-8265421
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3-1-6-s-auto/CP-AD-8265421/?Cr=59
ID: CP-AD-8277331
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-1-6-sport-v-bt/CP-AD-8277331/?Cr=60
ID: CP-AD-8279065
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-5-2-0-skyactiv-r-auto/CP-AD-8279065/?Cr=61
ID: GI-AD-491897
Enlace: https://www.chileautos.cl/vehiculos/detalles/mazda-cx-3-2020-r-2-0-aut/GI-AD-491897/?Cr=62
ID: GI-AD-426846
Enlace: https://www.chileautos.cl/vehiculos/detalles/mazda-6-2017-new-gt-2-2-aut-diesel/GI-AD-426846/?Cr=63
ID: CP-AD-8277601
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r/CP-AD-8277601/?Cr=64
ID: CL-AD-15491088
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5/CL-AD-15491088/?Cr=65
ID: CP-AD-8279205
Enlace: https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-5-2-0-skyactiv-gt-auto-4wd/CP-AD-8279205/?Cr=66
ID: CP-AD-8275759
Enlace: htt

ID: CL-AD-13317986
Enlace: https://www.chileautos.cl/vehiculos/detalles/2019-mazda-3-2-0-auto-skyactiv-g-v-sr/CL-AD-13317986/?Cr=125
ID: CP-AD-8274289
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-5-2-0-skyactiv-r-auto/CP-AD-8274289/?Cr=126
ID: CL-AD-15751863
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-30-2-0-v-auto-skyactive-g/CL-AD-15751863/?Cr=127
ID: CL-AD-15402501
Enlace: https://www.chileautos.cl/vehiculos/detalles/2015-mazda-cx-5-2-0-skyactiv-r/CL-AD-15402501/?Cr=128
ID: CP-AD-8254135
Enlace: https://www.chileautos.cl/vehiculos/detalles/2019-mazda-cx-3-2-0-r-auto-skyactive/CP-AD-8254135/?Cr=129
ID: CL-AD-15600436
Enlace: https://www.chileautos.cl/vehiculos/detalles/2019-mazda-cx-5-2-2-diesel-skyactiv-gt-auto-4wd/CL-AD-15600436/?Cr=130
ID: CL-AD-15631041
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-3-sport/CL-AD-15631041/?Cr=131
ID: CP-AD-8265509
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-5-cx5

ID: CL-AD-15490378
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-6-2-0-skyactiv-g-v-auto-cu/CL-AD-15490378/?Cr=189
ID: CL-AD-14687487
Enlace: https://www.chileautos.cl/vehiculos/detalles/2008-mazda-cx-7/CL-AD-14687487/?Cr=190
ID: CL-AD-14892551
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-5-2-2-diesel-skyactiv-gt-auto-4wd/CL-AD-14892551/?Cr=191
ID: CL-AD-14238121
Enlace: https://www.chileautos.cl/vehiculos/detalles/2007-mazda-6-2-0-r-16v-cuero/CL-AD-14238121/?Cr=192
ID: CL-AD-14419054
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-0-manual-skyactiv-r/CL-AD-14419054/?Cr=193
ID: CL-AD-15086836
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-0-manual-skyactiv-r/CL-AD-15086836/?Cr=194
ID: CL-AD-13524039
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-cx-9-3-7-gt-dvd-4wd/CL-AD-13524039/?Cr=195
ID: CL-AD-14357222
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-6-2-0-skyactiv

ID: CL-AD-15773135
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-5-2-0-skyactiv-r-auto-4wd/CL-AD-15773135/?Cr=255
ID: CL-AD-13539654
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx9-gt-awd-cx-9-gt-awd-automatica/CL-AD-13539654/?Cr=256
ID: CL-AD-13922241
Enlace: https://www.chileautos.cl/vehiculos/detalles/2022-mazda-cx-5-2-0-skyactiv-gt-auto-4wd/CL-AD-13922241/?Cr=257
ID: CL-AD-14735282
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-6-2-0-skyactiv-g-le-auto/CL-AD-14735282/?Cr=258
ID: CL-AD-15759026
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-1-6-s-ac/CL-AD-15759026/?Cr=259
ID: CL-AD-15335735
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-cx-5-2-0-syactiv-gt-auto-full-4wd/CL-AD-15335735/?Cr=260
ID: CL-AD-13735669
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-13735669/?Cr=261
ID: CL-AD-15640097
Enlace: https://www.chileautos.cl/vehiculos/detalle

...

ID: CL-AD-15213061
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-1-6-v-bt/CL-AD-15213061/?Cr=783
ID: CL-AD-15095631
Enlace: https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-1-6-s-auto/CL-AD-15095631/?Cr=784
ID: CL-AD-15581014
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3/CL-AD-15581014/?Cr=785
ID: CL-AD-15238935
Enlace: https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-sport/CL-AD-15238935/?Cr=786
ID: CL-AD-15297217
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-1-6-s/CL-AD-15297217/?Cr=787
ID: CL-AD-15497267
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-sport/CL-AD-15497267/?Cr=788
ID: CL-AD-14966005
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-14966005/?Cr=789
ID: CL-AD-15373824
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3-2-5-auto-skyactiv-g-gt/CL-AD-15373824/?Cr=790
ID: CL-AD-15636741
Enlace: https://www.ch

ID: CL-AD-15454817
Enlace: https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-2-0-skyactiv-g-v-sr/CL-AD-15454817/?Cr=850
ID: CL-AD-15473377
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-3-2-0-r-auto-skyactive/CL-AD-15473377/?Cr=851
ID: CL-AD-15555858
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-2-0-skyactiv-g-v-auto/CL-AD-15555858/?Cr=852
ID: CL-AD-15768221
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-5-2-0-skyactiv-r-auto-4wd/CL-AD-15768221/?Cr=853
ID: CL-AD-15599192
Enlace: https://www.chileautos.cl/vehiculos/detalles/2008-mazda-3-sport/CL-AD-15599192/?Cr=854
ID: CL-AD-15637964
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-2-1-5-v-skyactive-sport/CL-AD-15637964/?Cr=855
ID: CL-AD-15275720
Enlace: https://www.chileautos.cl/vehiculos/detalles/2010-mazda-6-2-0-v-auto/CL-AD-15275720/?Cr=856
ID: CL-AD-15830437
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-1-6-s-auto/CL-AD-15830437/?

ID: CL-AD-15691678
Enlace: https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15691678/?Cr=916
ID: CL-AD-14601608
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-3-2-0-skyactiv-g-v/CL-AD-14601608/?Cr=917
ID: CL-AD-15258632
Enlace: https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-9-2-5-gtx-auto-4wd/CL-AD-15258632/?Cr=918
ID: CL-AD-14848342
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-5-skyactiv-gtx-auto-4wd/CL-AD-14848342/?Cr=919
ID: CL-AD-15498832
Enlace: https://www.chileautos.cl/vehiculos/detalles/2012-mazda-bt-50/CL-AD-15498832/?Cr=920
ID: CL-AD-14963737
Enlace: https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-sport/CL-AD-14963737/?Cr=921
ID: CL-AD-15019766
Enlace: https://www.chileautos.cl/vehiculos/detalles/2020-mazda-bt-50-2-2-dsl-manual-sdx-4wd/CL-AD-15019766/?Cr=922
ID: CL-AD-15020694
Enlace: https://www.chileautos.cl/vehiculos/detalles/2021-mazda-bt-50-2-2-dsl-manual-sdx-hi-rider-4

ID: CL-AD-14034092
Enlace: https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx/CL-AD-14034092/?Cr=982
ID: CL-AD-14059021
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-bt-50-2-2-dsl-sdx/CL-AD-14059021/?Cr=983
ID: CL-AD-14560343
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-2-0-auto-skyactiv-g-v-sr/CL-AD-14560343/?Cr=984
ID: CL-AD-14660138
Enlace: https://www.chileautos.cl/vehiculos/detalles/2009-mazda-cx-9-3-7l-gt-autoawd/CL-AD-14660138/?Cr=985
ID: CL-AD-14668355
Enlace: https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-5-2-0-skyactiv-gt-auto-4wd/CL-AD-14668355/?Cr=986
ID: CL-AD-14770962
Enlace: https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-2-0-skyactiv-g-v-sr/CL-AD-14770962/?Cr=987
ID: CL-AD-15271111
Enlace: https://www.chileautos.cl/vehiculos/detalles/2013-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15271111/?Cr=988
ID: CL-AD-14837016
Enlace: https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-

AttributeError: 'NoneType' object has no attribute 'text'

In [67]:
autos = pd.DataFrame({'id': ids, 'enlace': links})
autos.to_csv('indices_enlaces.csv', index= False)

In [68]:
autos.head()

,id,enlace
0,CL-AD-14735282,https://www.chileautos.cl/vehiculos/detalles/2...
1,CL-AD-15499247,https://www.chileautos.cl/vehiculos/detalles/2...
2,CL-AD-15185232,https://www.chileautos.cl/vehiculos/detalles/2...
3,CL-AD-15158840,https://www.chileautos.cl/vehiculos/detalles/2...
4,CP-AD-8267126,https://www.chileautos.cl/vehiculos/detalles/2...


In [2]:
autos = pd.read_csv('indices_enlaces.csv')

## Preparación del Scraping

Luego de haber almacenado en un archivo CSV los ID y enlace de cada publicación, el siguiente segmento de código itera entre cada conjunto de observaciones y obtiene los datos solicitados en la primera parte del código, pero esta vez a toda la búsqueda de Mazda.

Comenzamos definiendo funciones que serán útiles para reducir la cantidad de código.

In [3]:
def obtener_valor(soup, clase):
    elemento = soup.find('div', class_=clase)
    if elemento:
        return elemento.text.strip()
    else:
        return None

def obtener_ubicacion(soup, clase):
    ubicacion = soup.find('p', class_=clase)
    if ubicacion:
        ubicacion_text = ubicacion.get_text(strip=True).split('\n')[0]
        return ubicacion_text
    else:
        return None

def obtener_motor(soup):
    litros_motor = obtener_valor(soup, 'col features-item-value features-item-value-motor-litros')
    if litros_motor:
        return float(litros_motor.split()[0])
    else:
        return None

def obtener_detalles(soup):
    detalles = soup.find_all('div', class_='key-details-item-title')
    if detalles:
        km = detalles[0].text.replace(' km', '').replace(',', '').strip()
        transmision = detalles[1].text.strip()
        combustible = detalles[2].text.strip()
        return km, transmision, combustible
    else:
        return None, None, None

def obtener_precio(soup):
    precio_elemento = obtener_valor(soup, 'col features-item-value features-item-value-precio')
    if precio_elemento:
        return int(precio_elemento.replace('$', '').replace(',', '').replace(' CLP', ''))
    else:
        return None

def parsear_autos_2(row, lista_autos, browser):
    id_autos = row['id']
    enlace = row['enlace']
    print(f'\nID auto: {id_autos}\n{enlace}')
    
    tiempo_pausa = random.randint(6, 8)
    print(f'Pausa aleatoria de {tiempo_pausa} segundos antes de la solicitud.')
    time.sleep(tiempo_pausa)

    browser.get(enlace)
    html = browser.page_source
    soup = bs(html, 'lxml')

    titulo = obtener_valor(soup, 'col-lg-8 col-sm-10')
    marca = obtener_valor(soup, 'col features-item-value features-item-value-marca')
    modelo = obtener_valor(soup, 'col features-item-value features-item-value-modelo')
    litros_motor = obtener_motor(soup)
    km, transmision, combustible = obtener_detalles(soup)
    precio = obtener_precio(soup)
    ubicacion = obtener_ubicacion(soup, 'mb-1')

    print(f'\nTítulo: {titulo}')
    print(f'El auto es marca {marca}')
    print(f'El auto marca {marca} es modelo {modelo}')
    print(f'Tiene un motor tamaño {litros_motor} litros')
    print(f'\nKilometraje: {km} km.\nTransmisión: {transmision}\nCombustible: {combustible}')
    print(f'El precio publicado es: {precio}')
    print(f'El vendedor es de {ubicacion}')

    info_auto = {
        'id': id_autos,
        'enlace': enlace,
        'titulo': titulo,
        'marca': marca,
        'modelo': modelo,
        'motor': litros_motor,
        'combustible': combustible,
        'kilometraje': km,
        'transmision': transmision,
        'precio': precio,
        'ubicacion': ubicacion
    }

    lista_autos.append(info_auto)

    return info_auto

# Scraping 

Como todo proceso liderado por humanos, este no está excento de errores. Considerando que al comienzo de la automatización del proceso se establecieron 1012 publicaciones, el código debía navegar por estas, lo que sumado al bajo tiempo establecido entre consultas, produjo un bloqueo de parte de la página. 

Esto fue solucionado mediante el uso de VPN, y solicitando el Scraping hasta el bloqueo. El proceso necesitó de 5 operaciones de Scraping, para finalmente obtener un solo DataFrame que contiene las 1012 publicaciones.

In [6]:
#Primer proceso de Scraping:

hora_actual = datetime.datetime.now().strftime("%H:%M:%S")
print("Hora de inicio:", hora_actual)
inicio = time.time()


browser = uc.Chrome()

lista_autos = []

for index, row in autos.iterrows():
    parsear_autos_2(row, lista_autos, browser)

browser.quit()


final = time.time()
hora_final = datetime.datetime.now().strftime("%H:%M:%S")
tiempo_total = (final - inicio) / 60

print('\nEl proceso ha terminado\n')
print(f'Hora de término: {hora_final}')
print(f'Pasaron {tiempo_total} minutos')


ID auto: CL-AD-14735282
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-6-2-0-skyactiv-g-le-auto/CL-AD-14735282/?Cr=0&gts=CL-AD-14735282&gtsSaleId=CL-AD-14735282&gtsViewType=showcase&rankingType=showcase
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: 2013 Mazda 6 2.0 Skyactiv-G LE Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA6
Tiene un motor tamaño 2.0 litros

Kilometraje: 103900 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10000000
El vendedor es de Puente Alto,Metropolitana de Santiago

ID auto: CL-AD-15499247
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-15499247/?Cr=0&gts=CL-AD-15499247&gtsSaleId=CL-AD-15499247&gtsViewType=showcase&rankingType=showcase
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: 2020 Mazda Bt-50 3.2 DSL SDX Auto Hi Rider 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 3.2 litros

Kilometraje


Título: Mazda 3 Sport 2016 2.5 SPORT GT SR GPS AT 5P
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 117677 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 9990000
El vendedor es de None

ID auto: GI-AD-493439
https://www.chileautos.cl/vehiculos/detalles/mazda-cx-3-2017-all-new-cx3-r-2-0-6mt/GI-AD-493439/?Cr=15
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: Mazda Cx-3 2017 ALL NEW CX3 R 2.0 6MT
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 106891 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 10990000
El vendedor es de Araucanía

ID auto: CP-AD-8270283
https://www.chileautos.cl/vehiculos/detalles/2022-mazda-cx-30-2-0-v-auto-skyactive-g/CP-AD-8270283/?Cr=16
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: 2022 Mazda Cx-30 2.0 V Auto Skyactive-G
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-


Título: 2015 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 91250 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 12790000
El vendedor es de Metropolitana deSantiago

ID auto: GI-AD-457157
https://www.chileautos.cl/vehiculos/detalles/mazda-2-sedan-2019-1-5-v-sedan-6mt-4p/GI-AD-457157/?Cr=34
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: Mazda 2 Sedan 2019 1.5 V SEDAN 6MT 4P
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 70519 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 10570000
El vendedor es de Metropolitana deSantiago

ID auto: CP-AD-8279984
https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-30-2-5-gtx-auto-skyactive-g-4wd/CP-AD-8279984/?Cr=35
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: 2021 Mazda Cx-30 2.5 GTX Auto Skyactive-G 4WD
El auto es marca M


Título: 2023 Mazda 3 2.5 Auto Skyactiv-G GTX HB
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.5 litros

Kilometraje: 6000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 24990000
El vendedor es de Metropolitana deSantiago

ID auto: GI-AD-466114
https://www.chileautos.cl/vehiculos/detalles/mazda-3-sport-2019-2-0-sport-at/GI-AD-466114/?Cr=53
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: Mazda 3 Sport 2019 2.0 SPORT AT
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 32115 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 14890000
El vendedor es de Metropolitana deSantiago

ID auto: CP-AD-8271002
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-3-2-5-skyactiv-g-gt-auto/CP-AD-8271002/?Cr=54
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2016 Mazda 3 2.5 Skyactiv-G GT Auto
El auto es marca MAZDA
El auto marca MAZD


Título: 2018 Mazda 3 2.5 Auto Skyactiv-G GT A. Bose
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.5 litros

Kilometraje: 50000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 14590000
El vendedor es de Viña del Mar, Valparaíso

ID auto: CP-AD-8278741
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-bt-50-dcab-sdx-4x4-3-2-at/CP-AD-8278741/?Cr=72
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: 2014 Mazda Bt-50 Dcab SDX 4x4 3.2 AT
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 139000 km.
Transmisión: Automática
Combustible: Diesel
El precio publicado es: 13980000
El vendedor es de Metropolitana deSantiago

ID auto: CL-AD-13742262
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-5-2-0-manual-skyactiv-r-4wd/CL-AD-13742262/?Cr=73
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: 2017 Mazda Cx-5 2.0 Manual Skyactiv R 4WD
El auto es

...


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-15498832
https://www.chileautos.cl/vehiculos/detalles/2012-mazda-bt-50/CL-AD-15498832/?Cr=920
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-14963737
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-sport/CL-AD-14963737/?Cr=921
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-15019766
ht


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-11652744
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-mx-6/CL-AD-11652744/?Cr=942
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-11731230
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3-sport/CL-AD-11731230/?Cr=943
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-15794607
htt


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-14715270
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-14715270/?Cr=963
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-14685253
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-6-2-0-skyactiv-g-v-auto-cu/CL-AD-14685253/?Cr=964
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
E


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-14560343
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-2-0-auto-skyactiv-g-v-sr/CL-AD-14560343/?Cr=984
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-14660138
https://www.chileautos.cl/vehiculos/detalles/2009-mazda-cx-9-3-7l-gt-autoawd/CL-AD-14660138/?Cr=985
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es d


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-15630628
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-0-manual-skyactiv-r/CL-AD-15630628/?Cr=1006
Pausa aleatoria de 4 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

ID auto: CL-AD-15673442
https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15673442/?Cr=1007
Pausa aleatoria de 5 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedo

In [9]:
#Crea el primer DataFrame con la información obtenida de 237 publicaciones

df = pd.DataFrame(lista_autos)
df.to_csv('0 a 237.csv', index= False)
df

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,CL-AD-14735282,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 6 2.0 Skyactiv-G LE Auto,MAZDA,MAZDA6,2.0,Bencina,103900,Automática,10000000.0,"Puente Alto,Metropolitana de Santiago"
1,CL-AD-15499247,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda Bt-50 3.2 DSL SDX Auto Hi Rider 4WD,MAZDA,BT-50,3.2,Diesel,92000,Automática,20900000.0,"Vitacura, Metropolitanade Santiago"
2,CL-AD-15185232,https://www.chileautos.cl/vehiculos/detalles/2...,2019 Mazda Bt-50 2.2 DSL Manual SDX 4WD,MAZDA,BT-50,2.2,Diesel,94000,Manual,21700000.0,"Galvarino, Araucanía"
3,CL-AD-15158840,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda 3 Sport,None,None,NaN,Bencina,25000,Automática,19800000.0,"Viña del Mar, Valparaíso"
4,CP-AD-8267126,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 2 1.5 V,MAZDA,MAZDA2,1.5,Bencina,61450,Manual,7390000.0,Metropolitana deSantiago
...,...,...,...,...,...,...,...,...,...,...,...
1007,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
1008,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
1009,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
1010,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None


In [5]:
#Segundo proceso de Scraping:

hora_actual = datetime.datetime.now().strftime("%H:%M:%S")
print("Hora de inicio:", hora_actual)
inicio = time.time()

browser = uc.Chrome()


lista_autos_238_1012 = []

for index, row in autos.iterrows():
    if index >= 236:
        print(f'Número de índice: {index}')
        parsear_autos_2(row, lista_autos_238_1012, browser)


browser.quit()

final = time.time()
hora_final = datetime.datetime.now().strftime("%H:%M:%S")
tiempo_total = (final - inicio) / 60

print('\nEl proceso ha terminado\n')
print(f'Hora de término: {hora_final}')
print(f'Pasaron {tiempo_total} minutos')

Hora de inicio: 11:04:21
Número de índice: 236

ID auto: CL-AD-12804826
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-2-1-5-v-ll-16/CL-AD-12804826/?Cr=232
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2014 Mazda 2 1.5 V LL 16
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA2
Tiene un motor tamaño 1.5 litros

Kilometraje: 123000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 7500000
El vendedor es de Estación Central,Metropolitana de Santiago
Número de índice: 237

ID auto: CL-AD-12996439
https://www.chileautos.cl/vehiculos/detalles/2021-mazda-3-sport/CL-AD-12996439/?Cr=233
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2021 Mazda 3 Sport
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 46000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 19800000
El vendedor es de Colina, Metropolitana deSantiago
Número de índice: 238

ID auto: CL-AD-1472


Título: 2019 Mazda Cx-5 2.0 Skyactiv R Auto 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 72000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 17500000
El vendedor es de Valdivia, Los Ríos
Número de índice: 254

ID auto: CP-AD-8267768
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-6-2-0-skyactiv-g-v-auto-ca/CP-AD-8267768/?Cr=250
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2019 Mazda 6 2.0 Skyactiv-G V Auto CA
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA6
Tiene un motor tamaño 2.0 litros

Kilometraje: 47137 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 15490000
El vendedor es de Los Lagos
Número de índice: 255

ID auto: CL-AD-15567959
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-hi-rider/CL-AD-15567959/?Cr=251
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2019 Mazda Bt-50 2.2 DSL 


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 273

ID auto: CL-AD-13435777
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-3-sport/CL-AD-13435777/?Cr=269
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 274

ID auto: CL-AD-14820855
https://www.chileautos.cl/vehiculos/detalles/2012-mazda-cx-7-r-2wd-cu/CL-AD-14820855/?Cr=270
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: Non


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 293

ID auto: CL-AD-13973160
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5/CL-AD-13973160/?Cr=289
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 294

ID auto: CL-AD-13858929
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-3-2-0-r-auto-skyactive/CL-AD-13858929/?Cr=290
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicad


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 313

ID auto: CL-AD-13305239
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r/CL-AD-13305239/?Cr=309
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 314

ID auto: CL-AD-15794765
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-3-1-6-s-auto/CL-AD-15794765/?Cr=310
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio public

...


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 934

ID auto: CL-AD-15510833
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-2/CL-AD-15510833/?Cr=930
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 935

ID auto: CL-AD-15542015
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-4wd/CL-AD-15542015/?Cr=931
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es:


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 955

ID auto: CL-AD-13712241
https://www.chileautos.cl/vehiculos/detalles/2022-mazda-bt-50-3-0-dsl-auto-4wd/CL-AD-13712241/?Cr=951
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 956

ID auto: CL-AD-15813912
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-15813912/?Cr=952
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combusti


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 975

ID auto: CL-AD-15225242
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-5-2-0-mav-v-auto/CL-AD-15225242/?Cr=971
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 976

ID auto: CL-AD-15802744
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3-sport/CL-AD-15802744/?Cr=972
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: 


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 995

ID auto: CL-AD-14931329
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-3-2-0-r-auto-skyactive/CL-AD-14931329/?Cr=991
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 996

ID auto: CL-AD-14938609
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-bt-50-2-2-dsl-sdx-4wd/CL-AD-14938609/?Cr=992
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
E

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción d

In [7]:
#Crea el segundo DataFrame con la información obtenida de 24 publicaciones

df_2 = pd.DataFrame(lista_autos_238_1012)
df_2.to_csv('238 a 262', index= False)
df_2

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción d

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,CL-AD-12804826,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda 2 1.5 V LL 16,MAZDA,MAZDA2,1.5,Bencina,123000,Manual,7500000.0,"Estación Central,Metropolitana de Santiago"
1,CL-AD-12996439,https://www.chileautos.cl/vehiculos/detalles/2...,2021 Mazda 3 Sport,None,None,NaN,Bencina,46000,Manual,19800000.0,"Colina, Metropolitana deSantiago"
2,CL-AD-14727911,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda Cx-9 3.7 GTX Auto 4WD,MAZDA,CX-9,3.7,Bencina,108000,Automática,12000000.0,"Macul, Metropolitana deSantiago"
3,CL-AD-13454837,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,123000,Automática,12900000.0,"Talagante, Metropolitanade Santiago"
4,CL-AD-13092688,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda Bt-50 2.2 DSL SDX 4WD,MAZDA,BT-50,2.2,Diesel,325000,Manual,18500000.0,"Temuco, Araucanía"
...,...,...,...,...,...,...,...,...,...,...,...
771,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
772,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
773,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
774,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\pablo\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\pablo\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción d

In [6]:
#Tercer proceso de Scraping:

hora_actual = datetime.datetime.now().strftime("%H:%M:%S")
print("Hora de inicio:", hora_actual)
inicio = time.time()


browser = uc.Chrome()

lista_autos_262_1012 = []

for index, row in autos.iterrows():
    if index >= 262:
        print(f'Número de índice: {index}')
        parsear_autos_2(row, lista_autos_262_1012, browser)

browser.quit()


final = time.time()
hora_final = datetime.datetime.now().strftime("%H:%M:%S")
tiempo_total = (final - inicio) / 60

print('\nEl proceso ha terminado\n')
print(f'Hora de término: {hora_final}')
print(f'Pasaron {tiempo_total} minutos')

Hora de inicio: 22:29:04
Número de índice: 262

ID auto: CL-AD-14735282
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-6-2-0-skyactiv-g-le-auto/CL-AD-14735282/?Cr=258
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2013 Mazda 6 2.0 Skyactiv-G LE Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA6
Tiene un motor tamaño 2.0 litros

Kilometraje: 103900 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10000000
El vendedor es de Puente Alto,Metropolitana de Santiago
Número de índice: 263

ID auto: CL-AD-15759026
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-1-6-s-ac/CL-AD-15759026/?Cr=259
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2014 Mazda 3 1.6 S AC
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 1.6 litros

Kilometraje: 91000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 8450000
El vendedor es de Vitacura, Metropolitanade Santiago
Número de 


Título: 2022 Mazda 3 2.0 Skyactiv-G S
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.0 litros

Kilometraje: 7500 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 16300000
El vendedor es de Las Condes,Metropolitana de Santiago
Número de índice: 280

ID auto: CL-AD-12667093
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-5-new-cx5-r-2-0-at/CL-AD-12667093/?Cr=276
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2017 Mazda Cx-5 New CX5 R 2.0 AT
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 39800 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 14990000
El vendedor es de La Reina, Metropolitanade Santiago
Número de índice: 281

ID auto: CL-AD-14678832
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-14678832/?Cr=277
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2014 Mazda Cx-5


Título: 2017 Mazda Cx-5 2.0 Manual Skyactiv R
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 123000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 12500000
El vendedor es de Coquimbo, Coquimbo
Número de índice: 298

ID auto: CL-AD-15257264
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15257264/?Cr=294
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2020 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 69300 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 17500000
El vendedor es de Bulnes, Ñuble
Número de índice: 299

ID auto: CL-AD-13358205
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-0-manual-skyactiv-r/CL-AD-13358205/?Cr=295
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2020 Mazda Cx-5 2.0 Manual Skyactiv R


Título: 2017 Mazda Cx-5
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 150003 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 12900000
El vendedor es de Huechuraba,Metropolitana de Santiago
Número de índice: 316

ID auto: CL-AD-15734990
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-sport/CL-AD-15734990/?Cr=312
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2014 Mazda 3 Sport
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 81000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10990000
El vendedor es de Colina, Metropolitana deSantiago
Número de índice: 317

ID auto: CL-AD-15216705
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-4wd/CL-AD-15216705/?Cr=313
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2019 Mazda Bt-50 2.2 DSL Manual SDX 4WD
El auto es marca


Título: 2017 Mazda Bt-50 3.2 DSL SDX Auto Hi Rider 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 3.2 litros

Kilometraje: 163000 km.
Transmisión: Automática
Combustible: Diesel
El precio publicado es: 19350000
El vendedor es de Huechuraba,Metropolitana de Santiago
Número de índice: 334

ID auto: CL-AD-14751152
https://www.chileautos.cl/vehiculos/detalles/2012-mazda-cx-7-gt-awd/CL-AD-14751152/?Cr=330
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2012 Mazda Cx-7 GT AWD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-7
Tiene un motor tamaño 2.3 litros

Kilometraje: 172000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 7200000
El vendedor es de Providencia,Metropolitana de Santiago
Número de índice: 335

ID auto: CL-AD-15527233
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-cx-3-2-0-manual-r-skyactive/CL-AD-15527233/?Cr=331
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2017 Ma

...


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 929

ID auto: CL-AD-15731772
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-cx-3-2-0-r-auto-skyactive-4wd/CL-AD-15731772/?Cr=925
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 930

ID auto: CL-AD-15210707
https://www.chileautos.cl/vehiculos/detalles/2017-mazda-3-2-5-skyactiv-g-gt-auto-a-bose/CL-AD-15210707/?Cr=926
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combu


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 950

ID auto: CL-AD-15698048
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-3-sport/CL-AD-15698048/?Cr=946
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 951

ID auto: CL-AD-12726256
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-hi-rider/CL-AD-12726256/?Cr=947
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El prec


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 970

ID auto: CL-AD-15591664
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-2-1-5-s-skyactive/CL-AD-15591664/?Cr=966
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 971

ID auto: CL-AD-14685178
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-3-sport/CL-AD-14685178/?Cr=967
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es:


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 990

ID auto: CL-AD-14668355
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-5-2-0-skyactiv-gt-auto-4wd/CL-AD-14668355/?Cr=986
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 991

ID auto: CL-AD-14770962
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-2-0-skyactiv-g-v-sr/CL-AD-14770962/?Cr=987
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: No


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 1010

ID auto: CL-AD-15630628
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-5-2-0-manual-skyactiv-r/CL-AD-15630628/?Cr=1006
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
Número de índice: 1011

ID auto: CL-AD-15673442
https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15673442/?Cr=1007
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible

In [7]:
#Crea el tercer DataFrame con la información obtenida de 251 publicaciones

df_3 = pd.DataFrame(lista_autos_262_1012)
df_3.to_csv('262 a 513.csv', index= False)
df_3

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,CL-AD-14735282,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 6 2.0 Skyactiv-G LE Auto,MAZDA,MAZDA6,2.0,Bencina,103900,Automática,10000000.0,"Puente Alto,Metropolitana de Santiago"
1,CL-AD-15759026,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda 3 1.6 S AC,MAZDA,MAZDA3,1.6,Bencina,91000,Manual,8450000.0,"Vitacura, Metropolitanade Santiago"
2,CL-AD-15335735,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda Cx-5 2.0 Syactiv GT Auto Full 4WD,MAZDA,CX-5,2.0,Bencina,88000,Automática,12499990.0,"Las Condes,Metropolitana de Santiago"
3,CL-AD-13735669,https://www.chileautos.cl/vehiculos/detalles/2...,2018 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,35000,Automática,17500000.0,"Concón, Valparaíso"
4,CL-AD-15640097,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda Cx-5 2.5 Skyactiv GT Auto 4WD,MAZDA,CX-5,2.5,Bencina,145000,Automática,15900000.0,"La Florida,Metropolitana de Santiago"
...,...,...,...,...,...,...,...,...,...,...,...
745,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
746,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
747,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
748,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None


In [4]:
#Cuarto proceso de Scraping:

hora_actual = datetime.datetime.now().strftime("%H:%M:%S")
print("Hora de inicio:", hora_actual)
inicio = time.time()

browser = uc.Chrome()


lista_autos_514_1012 = []

for index, row in autos.iterrows():
    if index >= 514:
        print('------------------------------')
        print(f'\nNúmero de índice: {index}')
        parsear_autos_2(row, lista_autos_514_1012, browser)

        
browser.quit()

final = time.time()
hora_final = datetime.datetime.now().strftime("%H:%M:%S")
tiempo_total = (final - inicio) / 60

print('\nEl proceso ha terminado\n')
print("La hora de inicio fue:", hora_actual)
print(f'Hora de término: {hora_final}')
print(f'Pasaron {tiempo_total} minutos')

Hora de inicio: 01:25:40
------------------------------

Número de índice: 514

ID auto: CL-AD-15623519
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-cx-5-2-0-syactiv-gt-auto-full-4wd/CL-AD-15623519/?Cr=510
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2013 Mazda Cx-5 2.0 Syactiv GT Auto Full 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 140000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10900000
El vendedor es de Providencia,Metropolitana de Santiago
------------------------------

Número de índice: 515

ID auto: CL-AD-13791638
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-13791638/?Cr=511
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2018 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 72000 km.
Transmisión: Automática
Combus


Título: 2016 Mazda 3 2.0 Skyactiv-G V SR
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.0 litros

Kilometraje: 128000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 10500000
El vendedor es de San Miguel,Metropolitana de Santiago
------------------------------

Número de índice: 531

ID auto: CL-AD-14684814
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-14684814/?Cr=527
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2018 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 62000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 17500000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 532

ID auto: CL-AD-15496498
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-4wd/CL-AD-15496498/?Cr


Título: 2019 Mazda 2 1.5 V Auto Skyactive Sport
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA2
Tiene un motor tamaño 1.5 litros

Kilometraje: 37000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 11690000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 547

ID auto: CL-AD-15672779
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-cx-5-2-0-skyactiv-r-auto-4wd/CL-AD-15672779/?Cr=543
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2013 Mazda Cx-5 2.0 Skyactiv R Auto 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 116000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10500000
El vendedor es de Providencia,Metropolitana de Santiago
------------------------------

Número de índice: 548

ID auto: CL-AD-15674210
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-auto-4w


Título: 2018 Mazda 3 Sport
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 82000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 13200000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 564

ID auto: CL-AD-13313482
https://www.chileautos.cl/vehiculos/detalles/2021-mazda-cx-5-2-0-skyactiv-gt-auto-4wd/CL-AD-13313482/?Cr=560
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2021 Mazda Cx-5 2.0 Skyactiv GT Auto 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 34000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 27500000
El vendedor es de Pucón, Araucanía
------------------------------

Número de índice: 565

ID auto: CL-AD-13445609
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-13445609/?Cr=561
Pausa aleatoria de 6 se


Título: 2017 Mazda 3 2.0 Skyactiv-G V
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.0 litros

Kilometraje: 133000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 10000000
El vendedor es de Concepción, Bío Bío
------------------------------

Número de índice: 581

ID auto: CL-AD-15843578
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-15843578/?Cr=577
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2019 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 76200 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 16990000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 582

ID auto: CL-AD-14601140
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-14601140/?Cr=578
Pausa al

...


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 936

ID auto: CL-AD-15541977
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-2/CL-AD-15541977/?Cr=932
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 937

ID auto: CL-AD-15848630
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-cx-5/CL-AD-15848630/?Cr=933
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: N


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 955

ID auto: CL-AD-13712241
https://www.chileautos.cl/vehiculos/detalles/2022-mazda-bt-50-3-0-dsl-auto-4wd/CL-AD-13712241/?Cr=951
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 956

ID auto: CL-AD-15813912
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-bt-50-3-2-dsl-sdx-auto-hi-rider-4wd/CL-AD-15813912/?Cr=952
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tama


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 974

ID auto: CL-AD-13865916
https://www.chileautos.cl/vehiculos/detalles/2007-mazda-6-2-0-v-16v-at/CL-AD-13865916/?Cr=970
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 975

ID auto: CL-AD-15225242
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-5-2-0-mav-v-auto/CL-AD-15225242/?Cr=971
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 993

ID auto: CL-AD-14837016
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-5-2-0-skyactiv-r-auto-4wd/CL-AD-14837016/?Cr=989
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None
------------------------------

Número de índice: 994

ID auto: CL-AD-15521783
https://www.chileautos.cl/vehiculos/detalles/2022-mazda-2-1-5-manual-gt-skyactive-sport/CL-AD-15521783/?Cr=990
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor ta


Título: None
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: None km.
Transmisión: None
Combustible: None
El precio publicado es: None
El vendedor es de None

El proceso ha terminado

La hora de inicio fue: 01:25:40
Hora de término: 03:22:49
Pasaron 117.13834639390309 minutos


In [5]:
#Crea el cuarto DataFrame con la información obtenida de 251 publicaciones


df_4 = pd.DataFrame(lista_autos_514_1012)
df_4.to_csv('514 a 765.csv', index= False)
df_4

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,CL-AD-15623519,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda Cx-5 2.0 Syactiv GT Auto Full 4WD,MAZDA,CX-5,2.0,Bencina,140000,Automática,10900000.0,"Providencia,Metropolitana de Santiago"
1,CL-AD-13791638,https://www.chileautos.cl/vehiculos/detalles/2...,2018 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,72000,Automática,18500000.0,"Padre Hurtado,Metropolitana de Santiago"
2,CL-AD-13728742,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,135000,Automática,11300000.0,"Las Condes,Metropolitana de Santiago"
3,CL-AD-13782374,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda 3 1.6 S Auto,MAZDA,MAZDA3,1.6,Bencina,105600,Automática,12000000.0,"Santiago, Metropolitanade Santiago"
4,CL-AD-14033865,https://www.chileautos.cl/vehiculos/detalles/2...,2017 Mazda 3 2.0 Skyactiv-G V Auto,MAZDA,MAZDA3,2.0,Bencina,83000,Automática,14750000.0,"Antofagasta, Antofagasta"
...,...,...,...,...,...,...,...,...,...,...,...
493,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
494,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
495,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None
496,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,None,None,None,NaN,None,None,None,NaN,None


In [4]:
#Quinto proceso de Scraping:

hora_actual = datetime.datetime.now().strftime("%H:%M:%S")
print("Hora de inicio:", hora_actual)
inicio = time.time()

browser = uc.Chrome()

lista_autos_766_1012 = []

for index, row in autos.iterrows():
    if index >= 766:
        print('------------------------------')
        print(f'\nNúmero de índice: {index}')
        parsear_autos_2(row, lista_autos_766_1012, browser)

browser.quit()

# Ahora, lista_autos contiene todos los datos recopilados
# Puedes procesar o guardar esta lista según tus necesidades

final = time.time()
hora_final = datetime.datetime.now().strftime("%H:%M:%S")
tiempo_total = (final - inicio) / 60

print('\nEl proceso ha terminado\n')
print("La hora de inicio fue:", hora_actual)
print(f'Hora de término: {hora_final}')
print(f'Pasaron {tiempo_total} minutos')

Hora de inicio: 03:26:38
------------------------------

Número de índice: 766

ID auto: CL-AD-13978687
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-auto-4wd/CL-AD-13978687/?Cr=762
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2014 Mazda Cx-5 2.0 Skyactiv R Auto 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 93772 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 11500000
El vendedor es de Padre Hurtado,Metropolitana de Santiago
------------------------------

Número de índice: 767

ID auto: CL-AD-14086119
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-auto/CL-AD-14086119/?Cr=763
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2014 Mazda Cx-5 2.0 Skyactiv R Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 133330 km.
Transmisión: Automática
Combustible: B


Título: 2011 Mazda 3 1.6 V
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 1.6 litros

Kilometraje: 115000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 6690000
El vendedor es de Maipú, Metropolitana deSantiago
------------------------------

Número de índice: 783

ID auto: CL-AD-15256643
https://www.chileautos.cl/vehiculos/detalles/2012-mazda-cx-7-r-2wd-cu/CL-AD-15256643/?Cr=779
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2012 Mazda Cx-7 R 2WD Cu
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-7
Tiene un motor tamaño 2.5 litros

Kilometraje: 115000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 8500000
El vendedor es de Copiapó, Atacama
------------------------------

Número de índice: 784

ID auto: CL-AD-14861323
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-cx-30/CL-AD-14861323/?Cr=780
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2020 Mazda Cx-30
E


Título: 2012 Mazda 2 1.5 V Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA2
Tiene un motor tamaño 1.5 litros

Kilometraje: 63000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 7000000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 800

ID auto: CL-AD-15512233
https://www.chileautos.cl/vehiculos/detalles/2016-mazda-cx-3-2-0-r-skyactive/CL-AD-15512233/?Cr=796
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2016 Mazda Cx-3 2.0 R Skyactive
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-3
Tiene un motor tamaño 2.0 litros

Kilometraje: 91000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 10500000
El vendedor es de Providencia,Metropolitana de Santiago
------------------------------

Número de índice: 801

ID auto: CL-AD-14478820
https://www.chileautos.cl/vehiculos/detalles/2020-mazda-3-sport/CL-AD-14478820/?Cr=797
Pausa aleatoria de 7 segundos ante


Título: 2017 Mazda Cx-3
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 100000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 11290000
El vendedor es de La Florida,Metropolitana de Santiago
------------------------------

Número de índice: 817

ID auto: CL-AD-13449634
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-bt-50-2-2-dsl-sdx-4wd/CL-AD-13449634/?Cr=813
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2015 Mazda Bt-50 2.2 DSL SDX 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 2.2 litros

Kilometraje: 208000 km.
Transmisión: Manual
Combustible: Diesel
El precio publicado es: 16499000
El vendedor es de Rauco, Maule
------------------------------

Número de índice: 818

ID auto: CL-AD-13238872
https://www.chileautos.cl/vehiculos/detalles/2013-mazda-3-1-6-sport-v-bt/CL-AD-13238872/?Cr=814
Pausa aleatoria de 6 segundos antes de la solicitud.

Tí


Título: 2015 Mazda 6 2.0 Skyactiv-G V Auto
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA6
Tiene un motor tamaño 2.0 litros

Kilometraje: 80500 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 11900000
El vendedor es de San Miguel,Metropolitana de Santiago
------------------------------

Número de índice: 834

ID auto: CL-AD-15337409
https://www.chileautos.cl/vehiculos/detalles/2011-mazda-cx-9/CL-AD-15337409/?Cr=830
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2011 Mazda Cx-9
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 100000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 10800000
El vendedor es de Las Condes,Metropolitana de Santiago
------------------------------

Número de índice: 835

ID auto: CL-AD-15829885
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-mx-5-2-0-manual-convertible/CL-AD-15829885/?Cr=831
Pausa aleatoria de 8 segundos a

...


Título: 2014 Mazda 2
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 69800 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 7650000
El vendedor es de Viña del Mar, Valparaíso
------------------------------

Número de índice: 935

ID auto: CL-AD-15542015
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-cx-5-2-0-skyactiv-r-4wd/CL-AD-15542015/?Cr=931
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2014 Mazda Cx-5 2.0 Skyactiv R 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo CX-5
Tiene un motor tamaño 2.0 litros

Kilometraje: 88000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 14000000
El vendedor es de La Reina, Metropolitanade Santiago
------------------------------

Número de índice: 936

ID auto: CL-AD-15541977
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-2/CL-AD-15541977/?Cr=932
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2014 


Título: 2019 Mazda Bt-50 2.2 DSL Manual SDX Hi Rider
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 2.2 litros

Kilometraje: 29000 km.
Transmisión: Manual
Combustible: Diesel
El precio publicado es: 28300000
El vendedor es de Concepción, Bío Bío
------------------------------

Número de índice: 952

ID auto: CL-AD-12726256
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx-hi-rider/CL-AD-12726256/?Cr=948
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2019 Mazda Bt-50 2.2 DSL Manual SDX Hi Rider
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 2.2 litros

Kilometraje: 29000 km.
Transmisión: Manual
Combustible: Diesel
El precio publicado es: 28300000
El vendedor es de Concepción, Bío Bío
------------------------------

Número de índice: 953

ID auto: CL-AD-15802024
https://www.chileautos.cl/vehiculos/detalles/2018-mazda-cx-3-2-0-r-auto-skyactive-4wd/CL-AD-15802024/?Cr=949
Pausa


Título: 2019 Mazda Bt-50 3.2 DSL SDX Auto Hi Rider 4WD
El auto es marca MAZDA
El auto marca MAZDA es modelo BT-50
Tiene un motor tamaño 3.2 litros

Kilometraje: 19000 km.
Transmisión: Automática
Combustible: Diesel
El precio publicado es: 28500000
El vendedor es de Talca, Maule
------------------------------

Número de índice: 968

ID auto: CL-AD-14685253
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-6-2-0-skyactiv-g-v-auto-cu/CL-AD-14685253/?Cr=964
Pausa aleatoria de 8 segundos antes de la solicitud.

Título: 2015 Mazda 6 2.0 Skyactiv-G V Auto CU
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA6
Tiene un motor tamaño 2.0 litros

Kilometraje: 144000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 9990000
El vendedor es de La Reina, Metropolitanade Santiago
------------------------------

Número de índice: 969

ID auto: CL-AD-15619213
https://www.chileautos.cl/vehiculos/detalles/2014-mazda-3-1-6-sport-v-auto-srf-bt/CL-AD-15619213/?Cr=965
Pau


Título: 2014 Mazda 2
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 77000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 6500000
El vendedor es de Osorno, Los Lagos
------------------------------

Número de índice: 985

ID auto: CL-AD-14410315
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-3-2-0-skyactiv-g-v/CL-AD-14410315/?Cr=981
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 2015 Mazda 3 2.0 Skyactiv-G V
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA3
Tiene un motor tamaño 2.0 litros

Kilometraje: 112610 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 9990000
El vendedor es de San Miguel,Metropolitana de Santiago
------------------------------

Número de índice: 986

ID auto: CL-AD-14034092
https://www.chileautos.cl/vehiculos/detalles/2019-mazda-bt-50-2-2-dsl-manual-sdx/CL-AD-14034092/?Cr=982
Pausa aleatoria de 8 segundos antes de la solicitud.

Tít


Título: 2011 Mazda 2
El auto es marca None
El auto marca None es modelo None
Tiene un motor tamaño None litros

Kilometraje: 189000 km.
Transmisión: Automática
Combustible: Bencina
El precio publicado es: 5850000
El vendedor es de Quilicura, Metropolitanade Santiago
------------------------------

Número de índice: 1001

ID auto: CL-AD-15467308
https://www.chileautos.cl/vehiculos/detalles/2015-mazda-2-1-5-s-ll/CL-AD-15467308/?Cr=997
Pausa aleatoria de 6 segundos antes de la solicitud.

Título: 2015 Mazda 2 1.5 S LL
El auto es marca MAZDA
El auto marca MAZDA es modelo MAZDA2
Tiene un motor tamaño 1.5 litros

Kilometraje: 85000 km.
Transmisión: Manual
Combustible: Bencina
El precio publicado es: 7200000
El vendedor es de Vitacura, Metropolitanade Santiago
------------------------------

Número de índice: 1002

ID auto: CL-AD-15302731
https://www.chileautos.cl/vehiculos/detalles/2012-mazda-3-1-6-sport-v/CL-AD-15302731/?Cr=998
Pausa aleatoria de 7 segundos antes de la solicitud.

Título: 

In [5]:
#Crea el cuarto DataFrame con la información obtenida de 246 publicaciones

df_5 = pd.DataFrame(lista_autos_766_1012)
df_5.to_csv('766 a 1012.csv', index= False)
df_5

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,CL-AD-13978687,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda Cx-5 2.0 Skyactiv R Auto 4WD,MAZDA,CX-5,2.0,Bencina,93772,Automática,11500000,"Padre Hurtado,Metropolitana de Santiago"
1,CL-AD-14086119,https://www.chileautos.cl/vehiculos/detalles/2...,2014 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,133330,Automática,12850000,"Ñuñoa, Metropolitana deSantiago"
2,CL-AD-14899458,https://www.chileautos.cl/vehiculos/detalles/2...,2017 Mazda Cx-5 2.0 Skyactiv GT Auto 4WD,MAZDA,CX-5,2.0,Bencina,105000,Automática,17950000,None
3,CL-AD-14276766,https://www.chileautos.cl/vehiculos/detalles/2...,2017 Mazda 6 2.5 Skyactiv-G GT Auto GPS,MAZDA,MAZDA6,2.5,Bencina,87000,Automática,15000000,"La Granja, Metropolitanade Santiago"
4,CL-AD-14293546,https://www.chileautos.cl/vehiculos/detalles/2...,2012 Mazda Cx-7 R 2WD,MAZDA,CX-7,2.5,Bencina,148000,Automática,8800000,"Puente Alto,Metropolitana de Santiago"
...,...,...,...,...,...,...,...,...,...,...,...
241,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,2018 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,57000,Automática,19690000,"Maipú, Metropolitana deSantiago"
242,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda 3 Sport,None,None,NaN,Bencina,146000,Manual,10500000,"Concón, Valparaíso"
243,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,2016 Mazda 3 2.0 Auto Skyactiv-G V SR,MAZDA,MAZDA3,2.0,Bencina,60000,Automática,11200000,"Colina, Metropolitana deSantiago"
244,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda Cx-5 2.0 Manual Skyactiv R,MAZDA,CX-5,2.0,Bencina,71000,Manual,17290000,"Puerto Varas, Los Lagos"


In [49]:
df_1 = pd.read_csv('0 a 237.csv')
df_2 = pd.read_csv('238 a 262.csv')
df_3 = pd.read_csv('262 a 513.csv')
df_4 = pd.read_csv('514 a 765.csv')
df_5 = pd.read_csv('766 a 1012.csv')

df_autos_final = pd.concat([df_1, df_2, df_3, df_4, df_5])
df_autos_final = df_autos_final.reset_index()
df_autos_final

,index,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,ubicacion
0,0,"CL-AD-14735282,https://www.chileautos.cl/vehic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"CL-AD-15499247,https://www.chileautos.cl/vehic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"CL-AD-15185232,https://www.chileautos.cl/vehic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"CL-AD-15158840,https://www.chileautos.cl/vehic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CP-AD-8267126,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 2 1.5 V,MAZDA,MAZDA2,1.5,Bencina,61450.0,Manual,7390000.0,Metropolitana deSantiago
...,...,...,...,...,...,...,...,...,...,...,...,...
1006,241,CL-AD-15490691,https://www.chileautos.cl/vehiculos/detalles/2...,2018 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,2.0,Bencina,57000.0,Automática,19690000.0,"Maipú, Metropolitana deSantiago"
1007,242,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda 3 Sport,NaN,NaN,NaN,Bencina,146000.0,Manual,10500000.0,"Concón, Valparaíso"
1008,243,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,2016 Mazda 3 2.0 Auto Skyactiv-G V SR,MAZDA,MAZDA3,2.0,Bencina,60000.0,Automática,11200000.0,"Colina, Metropolitana deSantiago"
1009,244,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda Cx-5 2.0 Manual Skyactiv R,MAZDA,CX-5,2.0,Bencina,71000.0,Manual,17290000.0,"Puerto Varas, Los Lagos"


## Solución de errores

En el proceso de obtención y exportación de los datos se cometieron algunos errores. Dentro de los más significativos tenemos: 

- Los datos, al momento de ser concatenados, no fueron reconocidos en un 100%, por lo que algunas observaciones tenían muchos NaN. Esto fue solucionado mediante el trabajo en Excel, donde se habían agregado caracteres especiales en las palabras que llevaban tilde, fueron removidos.

- Algunas observaciones contenían dentro de la ubicación, la localidad y la región, por lo que se creó una nueva columna llamada 'region' para ubicar esos datos por separado.

- (A futuro) En algunas publicaciones los datos de Marca y Modelo son inexistentes como tal, debido a que algunos vendedores no completan al 100% los datos solicitados al momento de vender. Sin embargo, en el título del anuncio se encuentran estos datos.

- (A futuro) Faltó considerar el año de cada vehículo como columna, sin embargo, está disponible dentro del título, por lo que puede ser fácilmente extraído.

***

Una vez reparado, cargamos el nuevo archivo .csv, resultando una tabla de 1012 columnas y 12 columnas.

In [63]:
df = pd.read_csv('Mazda ChileAutos.csv', sep = ';')
df

,id,enlace,titulo,marca,modelo,motor,combustible,kilometraje,transmision,precio,localidad,region
0,CL-AD-14735282,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 6 2.0 Skyactiv-G LE Auto,MAZDA,MAZDA6,20.0,Bencina,103900,Automatica,10000000.0,Puente Alto,NaN
1,CL-AD-15499247,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda Bt-50 3.2 DSL SDX Auto Hi Rider 4WD,MAZDA,BT-50,32.0,Diesel,92000,Automatica,20900000.0,Vitacura,NaN
2,CL-AD-15185232,https://www.chileautos.cl/vehiculos/detalles/2...,2019 Mazda Bt-50 2.2 DSL Manual SDX 4WD,MAZDA,BT-50,22.0,Diesel,94000,Manual,21700000.0,Galvarino,Araucania
3,CL-AD-15158840,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda 3 Sport,NaN,NaN,NaN,Bencina,25000,Automatica,19800000.0,Vina del Mar,Valparaiso
4,CP-AD-8267126,https://www.chileautos.cl/vehiculos/detalles/2...,2013 Mazda 2 1.5 V,MAZDA,MAZDA2,15.0,Bencina,61450,Manual,7390000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1007,CL-AD-15796914,https://www.chileautos.cl/vehiculos/detalles/2...,2015 Mazda 3 Sport,NaN,NaN,NaN,Bencina,146000,Manual,10500000,Concon,Valparaiso
1008,CL-AD-15771595,https://www.chileautos.cl/vehiculos/detalles/2...,2016 Mazda 3 2.0 Auto Skyactiv-G V SR,MAZDA,MAZDA3,20.0,Bencina,60000,Automatica,11200000,Colina,NaN
1009,CL-AD-15630628,https://www.chileautos.cl/vehiculos/detalles/2...,2020 Mazda Cx-5 2.0 Manual Skyactiv R,MAZDA,CX-5,20.0,Bencina,71000,Manual,17290000,Puerto Varas,Los Lagos
1010,CL-AD-15673442,https://www.chileautos.cl/vehiculos/detalles/2...,2021 Mazda Cx-5 2.0 Skyactiv R Auto,MAZDA,CX-5,20.0,Bencina,27500,Automatica,19750000,Santiago,NaN
